### Please update libraries and SDKs before we start

In [ ]:
!pip install --upgrade pip
!pip -q install sagemaker awscli boto3 --upgrade 

## PyTorch Model Listing with TorchServe on AWS Marketplace

In the notebook, we will use `TorchServe` and `Sagemaker SDK` to create a Docker image as a base, and re-use the docker image to list different model package products on AWS Marketplace with different `model data`.


### Clone the TorchServe repository and install torch-model-archiver

We'll use `torch-model-archiver` to create a model archive file (.mar). We'll use the .mar model archive file as the `model data` in listing different Pytorch models.

In [ ]:
!git clone https://github.com/pytorch/serve.git
!pip install serve/model-archiver/

### Create a boto3 session and get specify a role with SageMaker access

In [ ]:
import boto3, time, json
sess    = boto3.Session()
sm      = sess.client('sagemaker')
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

### Create an Amazon ECR registry through AWS CLI
Create a new docker container registry for your torchserve container images.

In [ ]:
registry_name = 'torchserve-base'
!aws ecr create-repository --repository-name {registry_name}

### Build a TorchServe Docker image and push it to Amazon ECR

In [ ]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}

#### Note: Remember to scan your docker image in Amazon ECR after you pushed the image.

![image info](./img/docker_image_scan.png)

### Download a PyTorch model and create a TorchServe archive
Let's start with Densenet-161

In [ ]:
!wget -q https://download.pytorch.org/models/densenet161-8d451a50.pth
    
model_file_name = 'densenet161'

!torch-model-archiver --model-name {model_file_name} \
--version 1.0 --model-file serve/examples/image_classifier/densenet_161/model.py \
--serialized-file densenet161-8d451a50.pth \
--extra-files serve/examples/image_classifier/index_to_name.json \
--handler image_classifier

!ls *.mar

#### Note: You can also replace with your own `pth` model file and create the `mar` file with `torch-model-archiver`

### Upload the generated densenet161.mar archive file to Amazon S3
Create a compressed tar.gz file from the densenet161.mar file since Amazon SageMaker expects that models are in a tar.gz file. 
Uploads the model to your default Amazon SageMaker S3 bucket under the models directory

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [ ]:
bucket_name = sagemaker_session.default_bucket()
prefix = 'torchserve'

!tar cvfz {model_file_name}.tar.gz densenet161.mar
!aws s3 cp {model_file_name}.tar.gz s3://{bucket_name}/{prefix}/models/

### Deploy endpoint and make prediction using Amazon SageMaker SDK, to make sure torchserve is actually working as the base image.

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name}.tar.gz'
sm_model_name = 'torchserve-densenet161'

torchserve_model = Model(model_data = model_data, 
                         image_uri = image,
                         role  = role,
                         predictor_cls=Predictor,
                         name  = sm_model_name)

In [ ]:
endpoint_name = 'torchserve-endpoint-' + sm_model_name + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = torchserve_model.deploy(instance_type='ml.m4.xlarge',
                                    initial_instance_count=1,
                                    endpoint_name = endpoint_name)

#### Test the TorchServe hosted model

In [ ]:
!wget -q https://s3.amazonaws.com/model-server/inputs/kitten.jpg    
file_name = 'kitten.jpg'
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = payload
    
response = predictor.predict(data=payload)
print(*json.loads(response), sep = '\n')

### Delete the endpoint

In [ ]:
predictor.delete_endpoint()

## Listing the Pytorch Model on Marketplace
In this section, we will work on listing two models with the same TorchServe base image. We only need to change the `model data` (the `mar` file) when creating another model package.

Let's first strt with the downloaded `Densenet-161`

In [ ]:
sm_model_name = 'torchserve-densenet161'
batch_inference_input_prefix = "batch-inference-input-data"
TRANSFORM_WORKDIR = "transform"

### Before we create the model package, Let's first test the batch transform on our side
First create the folder for transform input and download several images

In [ ]:
%%sh

# mkdir transform
cd transform
wget https://s3.amazonaws.com/model-server/inputs/kitten.jpg
wget https://s3.amazonaws.com/model-server/inputs/flower.jpg  

In [ ]:
transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

In [ ]:
transformer = sagemaker.transformer.Transformer(model_name=sm_model_name, instance_count=1, instance_type='ml.m4.xlarge',
                            strategy=None, assemble_with=None, output_path=None, sagemaker_session=sagemaker_session)

In [ ]:
transformer.transform(transform_input, content_type='image/jpeg')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

Congratualations! Batch transform succeed on our side!

### Create the model package

In [ ]:
from src.inference_specification import InferenceSpecification
import json

modelpackage_inference_specification = InferenceSpecification().get_inference_specification_dict(
    ecr_image=image,
    supports_gpu=True,
    supported_content_types=["image/jpeg", "image/png"],
    supported_mime_types=["application/json"])

# Specify the model data resulting from the previously completed training job
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]= model_data
print(json.dumps(modelpackage_inference_specification, indent=4, sort_keys=True))

In [ ]:
from src.modelpackage_validation_specification import ModelPackageValidationSpecification
import time

modelpackage_validation_specification = ModelPackageValidationSpecification().get_validation_specification_dict(
    validation_role = role,
    batch_transform_input = transform_input,
    input_content_type = "image/jpeg",
    output_content_type = "application/json",
    instance_type = "ml.c4.xlarge",
    output_s3_location = 's3://{}/{}'.format(sagemaker_session.default_bucket(), "/batch-inference-output-data"))

print(json.dumps(modelpackage_validation_specification, indent=4, sort_keys=True))

In [ ]:
model_package_name = sm_model_name + "-" + str(round(time.time()))
create_model_package_input_dict = {
    "ModelPackageName" : model_package_name,
    "ModelPackageDescription" : "Model of pre-trained DenseNet161",
    "CertifyForMarketplace" : True
}
create_model_package_input_dict.update(modelpackage_inference_specification)
create_model_package_input_dict.update(modelpackage_validation_specification)
print(json.dumps(create_model_package_input_dict, indent=4, sort_keys=True))

sm.create_model_package(**create_model_package_input_dict)

In [ ]:
while True:
    response = sm.describe_model_package(ModelPackageName=model_package_name)
    status = response["ModelPackageStatus"]
    print (status)
    if (status == "Completed" or status == "Failed"):
        print (response["ModelPackageStatusDetails"])
        break
    time.sleep(100)

### Let's use another model data to list another model product
Let's try with `Vgg-11` this time

You can also replace it with your own `pth` file

In [ ]:
!wget -q https://download.pytorch.org/models/vgg11-bbd30ac9.pth
    
model_file_name_vgg11 = 'vgg11'

!torch-model-archiver --model-name {model_file_name_vgg11} \
--version 1.0 --model-file serve/examples/image_classifier/vgg_11/model.py \
--serialized-file vgg11-bbd30ac9.pth \
--extra-files serve/examples/image_classifier/index_to_name.json \
--handler image_classifier

!ls *.mar

In [ ]:
prefix = 'torchserve'

!tar cvfz {model_file_name_vgg11}.tar.gz vgg11.mar
!aws s3 cp {model_file_name_vgg11}.tar.gz s3://{bucket_name}/{prefix}/models/

In [ ]:
model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name_vgg11}.tar.gz'
sm_model_name_vgg11 = 'torchserve-vgg11'

In [ ]:
modelpackage_inference_specification = InferenceSpecification().get_inference_specification_dict(
    ecr_image=image,
    supports_gpu=True,
    supported_content_types=["image/jpeg", "image/png"],
    supported_mime_types=["application/json"])

# Specify the model data resulting from the previously completed training job
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]= model_data
print(json.dumps(modelpackage_inference_specification, indent=4, sort_keys=True))


modelpackage_validation_specification = ModelPackageValidationSpecification().get_validation_specification_dict(
    validation_role = role,
    batch_transform_input = transform_input,
    input_content_type = "image/jpeg",
    output_content_type = "application/json",
    instance_type = "ml.c4.xlarge",
    output_s3_location = 's3://{}/{}'.format(sagemaker_session.default_bucket(), "/batch-inference-output-data"))

print(json.dumps(modelpackage_validation_specification, indent=4, sort_keys=True))

In [ ]:
model_package_name = sm_model_name_vgg11 + "-" + str(round(time.time()))
create_model_package_input_dict = {
    "ModelPackageName" : model_package_name,
    "ModelPackageDescription" : "Model of pre-trained VGG11",
    "CertifyForMarketplace" : True
}
create_model_package_input_dict.update(modelpackage_inference_specification)
create_model_package_input_dict.update(modelpackage_validation_specification)
print(json.dumps(create_model_package_input_dict, indent=4, sort_keys=True))

sm.create_model_package(**create_model_package_input_dict)

### Congratulations! You have succeeded creating two model packages for ML MarketPlace listing.

![image info](./img/listing_on_marketplace.png) 